In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
!which python

/Users/swethashankar/Documents/GitHub/MovieLens_DL/movielens/bin/python


In [3]:
credits_df= pd.read_csv("/Users/swethashankar/Documents/GitHub/MovieLens_DL/archive/credits.csv")
keywords_df= pd.read_csv("/Users/swethashankar/Documents/GitHub/MovieLens_DL/archive/keywords.csv")
links_df= pd.read_csv("/Users/swethashankar/Documents/GitHub/MovieLens_DL/archive/links_small.csv")
metadata_df= pd.read_csv("/Users/swethashankar/Documents/GitHub/MovieLens_DL/archive/movies_metadata.csv")
ratings_df= pd.read_csv("/Users/swethashankar/Documents/GitHub/MovieLens_DL/archive/ratings_small.csv")

/var/folders/dn/wkqtsjkx79xgl2mmhw0hd4000000gn/T/ipykernel_66166/2952159404.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df= pd.read_csv("/Users/swethashankar/Documents/GitHub/MovieLens_DL/archive/movies_metadata.csv")


In [4]:
print("Movies:", metadata_df.shape)
print("Credits:", credits_df.shape)
print("Keywords:", keywords_df.shape)
print("Ratings:", ratings_df.shape)
print("Links:", links_df.shape)

Movies: (45466, 24)
Credits: (45476, 3)
Keywords: (46419, 2)
Ratings: (100004, 4)
Links: (9125, 3)


### 1.1 Remove rows with null critical fields


In [5]:
print("credits:\n", credits_df.isnull().any())
print("\nkeywords:\n", keywords_df.isnull().any())
print("\nlinks small:\n", links_df.isnull().any())
print("\nmovies metadata:\n", metadata_df.isnull().any())
print("\nratings small:\n", ratings_df.isnull().any())

credits:
 cast    False
crew    False
id      False
dtype: bool

keywords:
 id          False
keywords    False
dtype: bool

links small:
 movieId    False
imdbId     False
tmdbId      True
dtype: bool

movies metadata:
 adult                    False
belongs_to_collection     True
budget                   False
genres                   False
homepage                  True
id                       False
imdb_id                   True
original_language         True
original_title           False
overview                  True
popularity                True
poster_path               True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
status                    True
tagline                   True
title                     True
video                     True
vote_average              True
vote_count                True
dtype: bool

ratings small:
 userId

In [6]:
print("links small:\n", links_df.isnull().sum())
print("movies metadata:\n", metadata_df.isnull().sum())

links small:
 movieId     0
imdbId      0
tmdbId     13
dtype: int64
movies metadata:
 adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64


In [7]:
links_cleaned= links_df.dropna(subset=["tmdbId"]).copy()
critical_fields=["imdb_id" , "original_language", "overview", "popularity","title"]
metadata_cleaned= metadata_df.dropna(subset=critical_fields)


In [8]:
print(links_cleaned.shape)
print(metadata_cleaned.shape)

(9112, 3)
(44481, 24)


### 1.2 Parse JSON columns into lists of names

In [9]:
print("Genres with suspicious values:")
print(metadata_df[~metadata_df["genres"].astype(str).str.startswith("[")]["genres"].unique())

print("\nKeywords with suspicious values:")
print(keywords_df[~keywords_df["keywords"].astype(str).str.startswith("[")]["keywords"].unique())

print("\nCast with suspicious values:")
print(credits_df[~credits_df["cast"].astype(str).str.startswith("[")]["cast"].unique())

print("\nCrew with suspicious values:")
print(credits_df[~credits_df["crew"].astype(str).str.startswith("[")]["crew"].unique())


Genres with suspicious values:
[]

Keywords with suspicious values:
[]

Cast with suspicious values:
[]

Crew with suspicious values:
[]


In [10]:
def parse_list_column(s, key="name"):
    try:
        return [d[key] for d in ast.literal_eval(s)]
    except Exception:
        return []

metadata_cleaned["genres"]= metadata_cleaned["genres"].apply(lambda x: parse_list_column(x, "name"))
keywords_df["keywords"]= keywords_df["keywords"].apply(lambda x: parse_list_column(x, "name"))
credits_df["cast"]= credits_df["cast"].apply(lambda x: parse_list_column(x, "name")[:5])  # top 5
credits_df["crew"]= credits_df["crew"].apply(lambda x: parse_list_column(x, "name"))


/var/folders/dn/wkqtsjkx79xgl2mmhw0hd4000000gn/T/ipykernel_66166/1864447569.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_cleaned["genres"]= metadata_cleaned["genres"].apply(lambda x: parse_list_column(x, "name"))


In [11]:
print(metadata_cleaned[["genres"]].head(3))
print(keywords_df[["keywords"]].head(3))
print(credits_df[["cast", "crew"]].head(3))

                         genres
0   [Animation, Comedy, Family]
1  [Adventure, Fantasy, Family]
2             [Romance, Comedy]
                                            keywords
0  [jealousy, toy, boy, friendship, friends, riva...
1  [board game, disappearance, based on children'...
2  [fishing, best friend, duringcreditsstinger, o...
                                                cast  \
0  [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...   
1  [Robin Williams, Jonathan Hyde, Kirsten Dunst,...   
2  [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...   

                                                crew  
0  [John Lasseter, Joss Whedon, Andrew Stanton, J...  
1  [Larry J. Franco, Jonathan Hensleigh, James Ho...  
2  [Howard Deutch, Mark Steven Johnson, Mark Stev...  


### 1.3 Convert data types (budget to int, release_date to datetime)

In [12]:
print(metadata_cleaned["budget"].dtypes)
print(metadata_cleaned["budget"].head(10))
print(metadata_cleaned["release_date"].dtypes)
print(metadata_cleaned["release_date"].head(10))

object
0    30000000
1    65000000
2           0
3    16000000
4           0
5    60000000
6    58000000
7           0
8    35000000
9    58000000
Name: budget, dtype: object
object
0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
5    1995-12-15
6    1995-12-15
7    1995-12-22
8    1995-12-22
9    1995-11-16
Name: release_date, dtype: object


In [13]:
def safe_to_numeric(series, dtype="float"):
    """Convert to numeric safely, coerce errors, and fill NaN with 0."""
    return pd.to_numeric(series, errors="coerce").fillna(0).astype(dtype)

# Convert numeric columns
metadata_cleaned["budget"]= safe_to_numeric(metadata_cleaned["budget"],dtype="int64")
metadata_cleaned["revenue"] =safe_to_numeric(metadata_cleaned["revenue"],dtype="int64")
metadata_cleaned["popularity"]= safe_to_numeric(metadata_cleaned["popularity"],dtype="float32")
metadata_cleaned["vote_average"] = safe_to_numeric(metadata_cleaned["vote_average"],dtype="float32")
metadata_cleaned["vote_count"] =safe_to_numeric(metadata_cleaned["vote_count"],dtype="int32")
metadata_cleaned["runtime"]= safe_to_numeric(metadata_cleaned["runtime"],dtype="float32")


/var/folders/dn/wkqtsjkx79xgl2mmhw0hd4000000gn/T/ipykernel_66166/2956698593.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_cleaned["budget"]= safe_to_numeric(metadata_cleaned["budget"],dtype="int64")
/var/folders/dn/wkqtsjkx79xgl2mmhw0hd4000000gn/T/ipykernel_66166/2956698593.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_cleaned["revenue"] =safe_to_numeric(metadata_cleaned["revenue"],dtype="int64")
/var/folders/dn/wkqtsjkx79xgl2mmhw0hd4000000gn/T/ipykernel_66166/2956698593.py:8: 

In [14]:
print(metadata_cleaned["budget"].head(10))
print(metadata_cleaned["budget"].dtype)
print("New NaN count:", metadata_cleaned["budget"].isna().sum()) # No new NaNs introduced

0    30000000
1    65000000
2           0
3    16000000
4           0
5    60000000
6    58000000
7           0
8    35000000
9    58000000
Name: budget, dtype: int64
int64
New NaN count: 0


In [15]:
# Convert release_date to datetime
metadata_cleaned["release_date"] = pd.to_datetime(metadata_cleaned["release_date"], errors="coerce")
print(metadata_cleaned["release_date"].dtype)
print(metadata_cleaned["release_date"].head(3))


datetime64[ns]
0   1995-10-30
1   1995-12-15
2   1995-12-22
Name: release_date, dtype: datetime64[ns]


/var/folders/dn/wkqtsjkx79xgl2mmhw0hd4000000gn/T/ipykernel_66166/306793262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_cleaned["release_date"] = pd.to_datetime(metadata_cleaned["release_date"], errors="coerce")


In [16]:
# Count invalid dates (become NaT after coercion)
invalid_dates = metadata_cleaned["release_date"].isna().sum()
total_rows = len(metadata_cleaned)

print(f"Invalid release_date rows: {invalid_dates} / {total_rows}")

Invalid release_date rows: 68 / 44481


In [17]:
# Drop rows with missing release_date
metadata_cleaned = metadata_cleaned.dropna(subset=["release_date"])

### 1.4 Drop duplicates and invalid IDs

In [18]:
print("Before: (metadata)")
print("Duplicates:", metadata_cleaned["id"].duplicated().sum())
print("Invalid IDs:", pd.to_numeric(metadata_cleaned["id"], errors="coerce").isna().sum())
print("Total rows:", len(metadata_cleaned))

# Clean
metadata_cleaned["id"] = pd.to_numeric(metadata_cleaned["id"], errors="coerce").astype("int64")
# metadata_cleaned = metadata_cleaned.dropna(subset=["id"])
metadata_cleaned = metadata_cleaned.drop_duplicates(subset="id", keep="first")

print("\nAfter:")
print("Duplicates:", metadata_cleaned["id"].duplicated().sum())
# print("Invalid IDs:", metadata_cleaned["id"].isna().sum())
print("Total rows:", len(metadata_cleaned))


Before: (metadata)
Duplicates: 30
Invalid IDs: 0
Total rows: 44413

After:
Duplicates: 0
Total rows: 44383


In [19]:
print("Before (credits):")
print("Duplicates:", credits_df["id"].duplicated().sum())
print("Invalid IDs:", pd.to_numeric(credits_df["id"], errors="coerce").isna().sum())
print("Total rows:", len(credits_df))

# Clean
credits_df["id"] = pd.to_numeric(credits_df["id"], errors="coerce").astype("int64")
# credits_df = credits_df.dropna(subset=["id"])
credits_df = credits_df.drop_duplicates(subset="id", keep="first")

print("\nAfter (credits):")
print("Duplicates:", credits_df["id"].duplicated().sum())
# print("Invalid IDs:", credits_df["id"].isna().sum())
print("Total rows:", len(credits_df))


Before (credits):
Duplicates: 44
Invalid IDs: 0
Total rows: 45476

After (credits):
Duplicates: 0
Total rows: 45432


In [20]:
print("Before (keywords):")
print("Duplicates:", keywords_df["id"].duplicated().sum())
print("Invalid IDs:", pd.to_numeric(keywords_df["id"], errors="coerce").isna().sum())
print("Total rows:", len(keywords_df))

# Clean
keywords_df["id"] = pd.to_numeric(keywords_df["id"], errors="coerce").astype("int64")
# keywords_df = keywords_df.dropna(subset=["id"])
keywords_df = keywords_df.drop_duplicates(subset="id", keep="first")

print("\nAfter (keywords):")
print("Duplicates:", keywords_df["id"].duplicated().sum())
# print("Invalid IDs:", keywords_df["id"].isna().sum())
print("Total rows:", len(keywords_df))


Before (keywords):
Duplicates: 987
Invalid IDs: 0
Total rows: 46419

After (keywords):
Duplicates: 0
Total rows: 45432


In [21]:
print("Before (ratings):")
print("Duplicates:", ratings_df.duplicated().sum())
print("Invalid movieIds:", pd.to_numeric(ratings_df["movieId"], errors="coerce").isna().sum())
print("Total rows:", len(ratings_df))

# Clean
ratings_df["movieId"] = pd.to_numeric(ratings_df["movieId"], errors="coerce").astype("int64")


Before (ratings):
Duplicates: 0
Invalid movieIds: 0
Total rows: 100004


In [22]:
print("Before (links):")
print("Duplicates:", links_cleaned.duplicated().sum())
print("Invalid movieId:", pd.to_numeric(links_cleaned["movieId"], errors="coerce").isna().sum())
print("Invalid imdbId:", pd.to_numeric(links_cleaned["imdbId"], errors="coerce").isna().sum())
print("Invalid tmdbId:", pd.to_numeric(links_cleaned["tmdbId"], errors="coerce").isna().sum())
print("Total rows:", len(links_cleaned))

# Clean
links_cleaned["movieId"] = pd.to_numeric(links_cleaned["movieId"], errors="coerce").astype("int64")
links_cleaned["imdbId"] = pd.to_numeric(links_cleaned["imdbId"], errors="coerce").astype("int64")
links_cleaned["tmdbId"] = pd.to_numeric(links_cleaned["tmdbId"], errors="coerce").astype("int64")

Before (links):
Duplicates: 0
Invalid movieId: 0
Invalid imdbId: 0
Invalid tmdbId: 0
Total rows: 9112


In [23]:
links_cleaned["tmdbId"].dtype

dtype('int64')

### 2.1 Build a unified movies frame

In [24]:
print(metadata_cleaned.columns)
movies_metadata= metadata_cleaned[["id","title","overview","genres","budget","runtime","release_date"]].copy()

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [25]:
print(keywords_df.columns)
credits_slim= credits_df[["id", "cast", "crew"]].copy()


Index(['id', 'keywords'], dtype='object')


In [26]:
print(credits_df.columns)
keywords_slim= keywords_df[["id", "keywords"]].copy()

Index(['cast', 'crew', 'id'], dtype='object')


In [27]:
movies= movies_metadata.merge(credits_slim, how="left", on="id").merge(keywords_slim, how="left", on="id")
movies.head()


,id,title,overview,genres,budget,runtime,release_date,cast,crew,keywords
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",30000000,81.0,1995-10-30,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...","[jealousy, toy, boy, friendship, friends, riva..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]",65000000,104.0,1995-12-15,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho...","[board game, disappearance, based on children'..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]",0,101.0,1995-12-22,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev...","[fishing, best friend, duringcreditsstinger, o..."
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]",16000000,127.0,1995-12-22,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...","[based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],0,106.0,1995-02-10,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N...","[baby, midlife crisis, confidence, aging, daug..."


In [28]:
# for column in ["genres","cast","crew","keywords"]:
#     print(movies[column].apply(type).value_counts())

In [29]:

for column in ["genres","cast","crew","keywords"]:
    movies[column]= movies[column].apply(lambda x: x if isinstance(x, list) else [])

movies.head(3)

,id,title,overview,genres,budget,runtime,release_date,cast,crew,keywords
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",30000000,81.0,1995-10-30,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...","[jealousy, toy, boy, friendship, friends, riva..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]",65000000,104.0,1995-12-15,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho...","[board game, disappearance, based on children'..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]",0,101.0,1995-12-22,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev...","[fishing, best friend, duringcreditsstinger, o..."


In [30]:
# Release year as numeric
movies["release_year"] = movies["release_date"].dt.year.fillna(0).astype("int32")

In [31]:
# # Add flag for "budget reported as zero"
# movies["budget_is_zero"] = (movies["budget"] == 0).astype(np.int8)

# # Safe log transform: log1p(0) = 0, handles large budgets smoothly
# movies["log_budget"] = np.log1p(movies["budget"])

### 2.2 Text embeddings (Sentence-Transformers)

In [32]:
# combine title+overview=text
def build_text(row):
    title=(row.get("title") or "").strip()
    overview=(row.get("overview") or "").strip()
    return f"{title}. {overview}" if overview else title

movies["text"]= movies.apply(build_text, axis=1)
movies.head(1)

,id,title,overview,genres,budget,runtime,release_date,cast,crew,keywords,release_year,text
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",30000000,81.0,1995-10-30,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...","[jealousy, toy, boy, friendship, friends, riva...",1995,"Toy Story. Led by Woody, Andy's toys live happ..."


In [33]:
# %pip install -q sentence-transformers

In [34]:
from sentence_transformers import SentenceTransformer

text_model_name= "sentence-transformers/all-MiniLM-L6-v2"
text_model = SentenceTransformer(text_model_name)

# Batch-encode for speed
texts= movies["text"].tolist()
text_embedding = text_model.encode(texts,batch_size=256,show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
print("Text embedding shape:", text_embedding.shape)

Batches:   0%|          | 0/174 [00:00<?, ?it/s]

Text embedding shape: (44383, 384)


In [35]:
print(text_embedding)

[[ 0.05570448  0.00022975  0.09581222 ...  0.03493924  0.04193446
   0.06821849]
 [ 0.07180642  0.08725393 -0.02754332 ... -0.01197139 -0.06999753
  -0.0225404 ]
 [-0.07744011  0.02054691 -0.00661253 ...  0.00445634 -0.03178686
   0.01753086]
 ...
 [-0.02480008 -0.01578687 -0.01370617 ... -0.02934398 -0.00814912
  -0.06585484]
 [-0.01156707  0.09120587 -0.06591202 ... -0.02868153 -0.00295109
  -0.03940088]
 [-0.0468686   0.01742156 -0.09697933 ... -0.06267873 -0.01851477
  -0.09517392]]


### 2.3 Categorical vocabularies → integer IDs

In [36]:
from collections import Counter

def build_vocab(series_of_lists, top_k=None, add_unknown=True):
    count = Counter()
    for l in series_of_lists:
        count.update([str(x).strip() for x in l if str(x).strip()])
    # sort by freq desc
    most = count.most_common(top_k) if top_k else count.most_common()
    idx = 0
    vocab = {}
    if add_unknown:
        vocab["<UNK>"] = idx; 
        idx += 1
    for token, j in most:
        if token not in vocab:
            vocab[token] = idx; 
            idx += 1
    return vocab

# Build vocabs
genres_vocab= build_vocab(movies["genres"],top_k=None)
cast_vocab= build_vocab(movies["cast"],top_k=5000)   # cap large lists
crew_vocab= build_vocab(movies["crew"],top_k=5000)
keywords_vocab= build_vocab(movies["keywords"], top_k=10000)

In [37]:
# print(keywords_vocab)

In [38]:
def map_list(l, vocab):
    return [vocab.get(str(x).strip(), vocab.get("<UNK>", 0)) for x in l if str(x).strip()]

movies["genres_ids"]= movies["genres"].apply(lambda x: map_list(x, genres_vocab))
movies["cast_ids"]= movies["cast"].apply(lambda x: map_list(x, cast_vocab)[:10])     # cap per-movie length
movies["crew_ids"]= movies["crew"].apply(lambda x: map_list(x, crew_vocab)[:10])
movies["keywords_ids"] = movies["keywords"].apply(lambda x: map_list(x, keywords_vocab)[:20])

# Quick peek
movies[["id","title","genres_ids","cast_ids","crew_ids","keywords_ids"]].head(3)

,id,title,genres_ids,cast_ids,crew_ids,keywords_ids
0,862,Toy Story,"[14, 2, 11]","[58, 799, 2183, 1976, 927]","[56, 1421, 662, 0, 0, 0, 0, 0, 0, 0]","[67, 1499, 337, 19, 93, 243, 6465, 8170, 1659]"
1,8844,Jumanji,"[9, 13, 11]","[49, 0, 453, 0, 1642]","[2611, 3729, 39, 4253, 0, 419, 0, 1880, 1172, ...","[3147, 357, 866, 2648, 3148, 1840]"
2,15602,Grumpier Old Men,"[4, 2]","[110, 111, 421, 258, 129]","[0, 2321, 2321, 4254]","[1187, 143, 16, 8171]"


### 2.4 Numeric features normalization

In [39]:
from sklearn.preprocessing import StandardScaler

num_columns = ["budget", "runtime", "release_year"]
scaler= StandardScaler()
num_features = scaler.fit_transform(movies[num_columns].values.astype("float32"))
print("Numeric features shape:",num_features.shape)

Numeric features shape: (44383, 3)


In [40]:
num_features

array([[ 1.4574289 , -0.37442946,  0.13250543],
       [ 3.4438074 ,  0.23886582,  0.13250543],
       [-0.24518108,  0.15887079,  0.13250543],
       ...,
       [-0.24518108, -0.13444434,  0.4640645 ],
       [-0.24518108, -0.21443938, -3.1001956 ],
       [-0.24518108, -0.5344195 ,  1.0442929 ]], dtype=float32)

### 2.5 Package features + save artifacts

In [41]:
import pickle, os

# setup
features_dir = "./features_artifacts"
os.makedirs(features_dir, exist_ok=True)

tmdb_ids = movies["id"].values.astype("int64")

# Save numerical/text features
np.savez_compressed(
    os.path.join(features_dir, "content_features_npz.npz"),
    tmdb_id = tmdb_ids,
    text_emb = text_embedding.astype("float32"),
    num_feats = num_features.astype("float32"),
)

# save auxiliary artifacts
artifacts = {
    "genres_vocab": genres_vocab,
    "cast_vocab": cast_vocab,
    "crew_vocab": crew_vocab,
    "keywords_vocab": keywords_vocab,
    "scaler_num": scaler,
    "text_model": text_model_name,
    "num_cols": num_columns,
}
with open(os.path.join(features_dir, "content_feature_artifacts.pkl"), "wb") as f:
    pickle.dump(artifacts, f)

# save categorical ID lists
id_lists = movies[["id","genres_ids","cast_ids","crew_ids","keywords_ids"]].copy()
id_lists.to_parquet(os.path.join(features_dir, "categorical_id_lists.parquet"), index=False)

print("Saved:", os.listdir(features_dir))


Saved: ['movie_embeddings_finetuned.npz', 'content_feature_artifacts.pkl', 'movie_faiss_index_flat.index', 'movie_embeddings.npz', 'categorical_id_lists.parquet', 'content_features_npz.npz']


# Content-Based Model
### 3.0 Load Artifacts

In [42]:
# load the features
npz = np.load(os.path.join(features_dir,"content_features_npz.npz"))
tmdb_ids= npz["tmdb_id"]          # [N]
text_emb= npz["text_emb"]         # [N, 384] for MiniLM
num_feats= npz["num_feats"] 

In [43]:
# Load categorical IDs, vocabularies, scalers, metadata
id_lists= pd.read_parquet(os.path.join(features_dir,"categorical_id_lists.parquet"))


In [44]:
with open(os.path.join(features_dir, "content_feature_artifacts.pkl"), "rb") as f:
    art= pickle.load(f)

genres_vocab= art["genres_vocab"]
cast_vocab= art["cast_vocab"]
crew_vocab= art["crew_vocab"]
keywords_vocab= art["keywords_vocab"]

# recover vocab sizes
V_G= max(genres_vocab.values())+1
V_C= max(cast_vocab.values())+1
V_R= max(crew_vocab.values())+1
V_K= max(keywords_vocab.values())+1

print(text_emb.shape, num_feats.shape, id_lists.shape)

(44383, 384) (44383, 3) (44383, 5)


### 3.1 Dataset: anchors + positives + negatives

In [45]:
import torch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
import random

In [46]:
# # Build inverted indices for e.g. movies and genres
def build_token_index(series_of_lists):
    token_to_idx= defaultdict(set)
    for idx, lst in enumerate(series_of_lists):
        for token in lst:
            token_to_idx[int(token)].add(idx)
    return token_to_idx

# for e.g. inv_genres = {1: {0,2}, 2: {0}, 3: {1}, 4: {1}, 5: {2}}
# Meaning: Genre 1 appears in movies 0 and 2.
# The argument series_of_lists is aligned with the row index of movies

In [47]:
genre_ids= id_lists["genres_ids"].tolist()
cast_ids= id_lists["cast_ids"].tolist()
crew_ids= id_lists["crew_ids"].tolist()
keywords_ids= id_lists["keywords_ids"].to_list()

inv_genres= build_token_index(genre_ids)
# inv_cast= build_token_index(cast_ids)
# inv_crew= build_token_index(crew_ids)
inv_keywords= build_token_index(keywords_ids)

In [48]:
def candidates_from_inv(idx):
    # make a union of same-genre or same-keyword neighbors (exclude/drop self)
    candidates = set()
    for g in genre_ids[idx]:
        candidates |= inv_genres.get(g, set())
    for k in keywords_ids[idx]:
        candidates |= inv_keywords.get(k, set())
    candidates.discard(idx)  # drop self
    return list(candidates)

# Example if movie 0 is “Rush Hour” (Action, Comedy), positives will be any other Action or Comedy movies.

In [49]:
class MoviePairs(Dataset):
    def __init__(self, text_emb, num_feats, genre_ids, cast_ids, crew_ids, keywords_ids):
        # Initializes the dataset with text embeddings, numeric features, and categorical ID lists.
        # Also sets up references to genre, cast, crew, and keyword ID lists, and stores the total number of movies.
        self.num_feats = num_feats
        self.text_emb = text_emb
        self.g = genre_ids
        self.c = cast_ids
        self.r = crew_ids
        self.k = keywords_ids
        self.N = len(text_emb)
        self.all_idx = list(range(self.N))

    def __len__(self):
        # Returns the total number of movies (dataset size).
        return self.N
    
    def __getitem__(self, i):
        # Given an index i (anchor movie), selects:
        # - a positive sample: another movie sharing at least one genre or keyword (if available), otherwise random
        # - a negative sample: a random movie different from anchor and positive
        # Returns a dictionary with features for anchor, positive, and negative movies.
        anchor = i
        pos_candidates = candidates_from_inv(anchor)
        if pos_candidates:
            positive = random.choice(pos_candidates)
        else:
            positive = random.randrange(self.N)

        negative = random.randrange(self.N)
        while negative == anchor or negative == positive:
            negative = random.randrange(self.N)
    
        return {
            "anchor_text": self.text_emb[anchor],
            "anchor_num": self.num_feats[anchor],
            "anchor_g": list(self.g[anchor]),
            "anchor_c": list(self.c[anchor]),
            "anchor_r": list(self.r[anchor]),
            "anchor_k": list(self.k[anchor]),
            "positive_text": self.text_emb[positive],
            "positive_num": self.num_feats[positive],
            "positive_g": list(self.g[positive]),
            "positive_c": list(self.c[positive]),
            "positive_r": list(self.r[positive]),
            "positive_k": list(self.k[positive]),
            "negative_text": self.text_emb[negative],
            "negative_num": self.num_feats[negative],
            "negative_g": list(self.g[negative]),
            "negative_c": list(self.c[negative]),
            "negative_r": list(self.r[negative]),
            "negative_k": list(self.k[negative]),
        }


In [50]:
def collate(batch):
    # Convert to tensors; lists stay as Python lists (pad inside the model)
    to_tensor= lambda x: torch.tensor(np.stack(x), dtype=torch.float32)
    pack= {}
    for prefix in ["anchor", "positive", "negative"]:
        pack[f"{prefix}_text"]= to_tensor([item[f"{prefix}_text"] for item in batch])
        pack[f"{prefix}_num"]= to_tensor([item[f"{prefix}_num"] for item in batch])
        for feat in ["g","c","r","k"]:
            pack[f"{prefix}_{feat}"]= [item[f"{prefix}_{feat}"] for item in batch]
    return pack

# DataLoader with batching (256) and shuffling (of each epoch)- creates a dataset of triplets (anchor, positive, negative)
dataset= MoviePairs(text_emb, num_features, genre_ids, cast_ids, crew_ids, keywords_ids)
dataloader= DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0, collate_fn=collate)   
next(iter(dataloader)).keys()

dict_keys(['anchor_text', 'anchor_num', 'anchor_g', 'anchor_c', 'anchor_r', 'anchor_k', 'positive_text', 'positive_num', 'positive_g', 'positive_c', 'positive_r', 'positive_k', 'negative_text', 'negative_num', 'negative_g', 'negative_c', 'negative_r', 'negative_k'])

### 3.2 Model: embeddings + mean pooling → 128-D

In [51]:
import torch.nn as nn
import torch.nn.functional as F

class BagEmbed(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

    def forward(self, lists_of_ids):
        B= len(lists_of_ids)
        maxlen= max((len(l) for l in lists_of_ids), default=1)
        device= self.embedding.weight.device
        input_tensor= torch.zeros((B, maxlen), dtype=torch.long, device=device)
        lens=[]
        for i, l in enumerate(lists_of_ids):
            lens.append(len(l))
            if len(l) > 0:
                input_tensor[i, :len(l)] = torch.tensor(l, dtype=torch.long, device=device)
        embeds= self.embedding(input_tensor)  # [B, maxlen, D]
        mask= (input_tensor != 0).float().unsqueeze(-1)  # [B, maxlen, 1]
        pooled= (embeds * mask).sum(dim=1) / (mask.sum(dim=1).clamp_min(1.0))  # mean pool
        return pooled  # [B, D]


In [52]:
class ContentTower(nn.Module):
    def __init__(self, dims, vocab_sizes):
        super().__init__()
        self.genres= BagEmbed(vocab_sizes["genres"], dims["cat"])
        self.cast= BagEmbed(vocab_sizes["cast"], dims["cat"])
        self.crew= BagEmbed(vocab_sizes["crew"], dims["cat"])
        self.keywords= BagEmbed(vocab_sizes["keywords"], dims["cat"])

        d_in= dims["text"] + dims["num"] + 4 * dims["cat"]
        self.mlp= nn.Sequential(
            nn.Linear(d_in, 256), nn.ReLU(),
            nn.Linear(256, dims["out"])
        )

    def forward(self, text_emb, num_feat, g_ids, c_ids, r_ids, k_ids):
        g_emb= self.genres(g_ids)
        c_emb= self.cast(c_ids)
        r_emb= self.crew(r_ids)
        k_emb= self.keywords(k_ids)
        x= torch.cat([text_emb, num_feat, g_emb, c_emb, r_emb, k_emb], dim=1)
        z= F.normalize(self.mlp(x), p=2, dim=1)
        return z


In [53]:
dims= {"text": text_emb.shape[1], "num": num_features.shape[1], "cat": 64, "out": 128}
vocab_sizes= {"genres": V_G, "cast": V_C, "crew": V_R, "keywords": V_K}
model= ContentTower(dims, vocab_sizes).to("cuda" if torch.cuda.is_available() else "cpu")

### 3.3 Loss: choose one (cosine or triplet)

In [54]:
# Option A: Triplet Margin Loss
# loss_fn= nn.TripletMarginLoss(margin=0.2, p=2)

# Option B: Custom Cosine Pair Loss (cosine similarity loss)
def cosine_pair_loss(a, p, n, margin=0.2):
    # cos sim in [-1, 1]
    cos_ap= (a*p).sum(dim=1)  # [B]
    cos_an= (a*n).sum(dim=1)
    # hinge: (margin- ap+ an)
    loss= torch.clamp(margin - cos_ap + cos_an, min=0.0).mean()
    return loss

### 3.4 TRAINING LOOP

In [55]:
import math

device= "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer= torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

EPOCHS=3
for epoch in range(EPOCHS):
    model.train()
    total_loss= 0.0
    for step, batch in enumerate(dataloader, 1):
        a= model(
            batch["anchor_text"].to(device),
            batch["anchor_num"].to(device),
            batch["anchor_g"],
            batch["anchor_c"],
            batch["anchor_r"],
            batch["anchor_k"],
        )
        p= model(
            batch["positive_text"].to(device),
            batch["positive_num"].to(device),
            batch["positive_g"],
            batch["positive_c"],
            batch["positive_r"],
            batch["positive_k"],
        )
        n= model(
            batch["negative_text"].to(device),
            batch["negative_num"].to(device),
            batch["negative_g"],
            batch["negative_c"],
            batch["negative_r"],
            batch["negative_k"],
        )
        loss= cosine_pair_loss(a, p, n, margin=0.2) # or triplet(a,p,n)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if step % 50 == 0:
            avg_loss= total_loss / step
            print(f"Epoch {epoch+1} Step {step} Avg Loss: {avg_loss:.4f}")
            total_loss= 0.0
# End of epoch

# Save the trained model weights
os.makedirs("./checkpoints", exist_ok=True)
torch.save(model.state_dict(), "./checkpoints/content_tower.pt")
print("Model saved to ./checkpoints/content_tower.pt")
        


Epoch 1 Step 50 Avg Loss: 0.1357
Epoch 1 Step 100 Avg Loss: 0.0591
Epoch 1 Step 150 Avg Loss: 0.0368
Epoch 2 Step 50 Avg Loss: 0.1069
Epoch 2 Step 100 Avg Loss: 0.0537
Epoch 2 Step 150 Avg Loss: 0.0360
Epoch 3 Step 50 Avg Loss: 0.1021
Epoch 3 Step 100 Avg Loss: 0.0533
Epoch 3 Step 150 Avg Loss: 0.0350
Model saved to ./checkpoints/content_tower.pt


### 3.5 Export the movie embedding matrix

In [56]:
model.eval()
B= 2048  # batch size for inference
all_embeddings= []
with torch.no_grad():
    for i in range(0, len(tmdb_ids), B):
        Z = model(
            torch.tensor(text_emb[i:i+B], dtype=torch.float32, device=device),
            torch.tensor(num_features[i:i+B], dtype=torch.float32, device=device),
            genre_ids[i:i+B],
            cast_ids[i:i+B],
            crew_ids[i:i+B],
            keywords_ids[i:i+B]
        )
        all_embeddings.append(Z.cpu().numpy())

movie_embeddings= np.vstack(all_embeddings).astype("float32")  # [N, 128]
print("Movie embeddings shape:", movie_embeddings.shape)

np.savez_compressed(os.path.join(features_dir, "movie_embeddings.npz"), tmdb_id=tmdb_ids, emb=movie_embeddings)

Movie embeddings shape: (44383, 128)


### 3.6 Quick retrieval test (no FAISS yet)

In [57]:
def topk_neighbors(seed_idx, k=10):
    sims= movie_embeddings @ movie_embeddings[seed_idx] # since both [D], [N, D] are L2-normalized, cosine=dot product (=[N])
    sims[seed_idx] = -1.0                   # exclude self
    nbrs= np.argpartition(-sims, k)[:k]
    nbrs= nbrs[np.argsort(-sims[nbrs])]
    return nbrs, sims[nbrs]

seed_idx= int(np.where(tmdb_ids==862)[0][0]) # find index of TMDB ID 862 (Toy Story)
nbrs, scores= topk_neighbors(seed_idx, k=10)
print("Neighbors of TMDB ID 862 (Toy Story):", tmdb_ids[nbrs], "Scores:", scores)

Neighbors of TMDB ID 862 (Toy Story): [   863    771 196633   9969  37556  44157  31309  16508  36251  73541] Scores: [0.9179871  0.8985822  0.89654386 0.8953173  0.89306736 0.88762194
 0.8855413  0.88159513 0.8811209  0.8805306 ]


In [58]:
# Map to titles for a quick eyeball check
# Build once:
id_to_title = dict(zip(movies["id"].astype(int).values, movies["title"].astype(str).values))

for i, s in zip(nbrs, scores):
    mid = int(tmdb_ids[i])
    print(f"{mid:>7}  {id_to_title.get(mid, '?')}  cos={float(s):.3f}")


    863  Toy Story 2  cos=0.918
    771  Home Alone  cos=0.899
 196633  Botsman i Popugay  cos=0.897
   9969  Deck the Halls  cos=0.895
  37556  Charlie Brown's Christmas Tales  cos=0.893
  44157  How To Frame A Figg  cos=0.888
  31309  Mr. Bug Goes to Town  cos=0.886
  16508  Doug's 1st Movie  cos=0.882
  36251  Switching Goals  cos=0.881
  73541  You Should Meet My Son  cos=0.881


In [59]:
# Alignment & slicing gotchas (very important)
# Make sure the arrays line up 1:1

assert movie_embeddings.shape[0] == len(tmdb_ids)
genres_ids= id_lists["genres_ids"].tolist()
cast_ids= id_lists["cast_ids"].tolist()
crew_ids= id_lists["crew_ids"].tolist()
keywords_ids= id_lists["keywords_ids"].tolist()

# Sanity: the seed truly corresponds to TMDB 862
assert tmdb_ids[seed_idx] == 862


In [60]:
# Brave sanity check

In [61]:
# data= np.load("./features_artifacts/movie_embeddings.npz")
# emb= data["emb"]
# tmdb= data["tmdb_id"]  

# # Load ID -> title map
# with open("./features_artifacts/content_feature_artifacts.pkl", "rb") as f:
#     art= pickle.load(f)

# print(type(art))
# if isinstance(art, dict):
#     print("Dict keys:", list(art.keys())[:10])
# elif hasattr(art, "head"):  # probably a pandas DataFrame
#     print(art.head())
# else:
#     print("First element:", str(art)[:500])


# def show_neighbors(seed_id, topk=10):
#     idx = np.where(tmdb == seed_id)[0][0]
#     sims = emb @ emb[idx]  # cosine similarity since L2-normalized
#     nn_idx = sims.argsort()[::-1][1:topk+1]  # exclude self
#     print("Seed:", id_to_title.get(int(seed_id), str(seed_id)))
#     for j in nn_idx:
#         mid= int(tmdb_ids[j])
#         print(f" {mid:>7} {id_to_title.get(mid, '?'):40s} cos={sims[j]:.3f}")


# # SPot check across genres
# show_neighbors(862) # Toy Story
# show_neighbors(550) # Fight Club → should find similar dark dramas
# show_neighbors(603) # The Matrix → should find sci-fi / action
# show_neighbors(13) # Forrest Gump → should find dramas/romance

In [62]:
# def neighbor_overlap(idx, field, topk=10):
#     """Compute how often the neighbors share at least one genre/keyword with the seed."""
#     sims= emb @ emb[idx]
#     nn_idx= sims.argsort()[::-1][1:topk+1]  # exclude self
#     target_set= set(id_lists.iloc[idx][field])
#     overlaps=[]
#     for j in nn_idx:
#         nbr_set= set(id_lists.iloc[j][field])
#         overlaps.append(1 if target_set & nbr_set else 0)

#     return np.mean(overlaps)
    
# # Sample 100 seeds at random
# random.seed(42)
# sample_idx= random.choices(range(len(tmdb)), k=100)
# mean_genre_overlap= np.mean([neighbor_overlap(i, "genres_ids", topk=10) for i in sample_idx])
# print(f"Mean genre overlap in top 10 neighbors: {mean_genre_overlap:.3f}")

In [63]:
# Mean genre overlap in top 10 neighbors: 0.920
# This is very high (92%) → almost all top-10 neighbors share at least one genre.

# Confirms the embeddings are clustering movies correctly in the vector space.

In [64]:
# id_to_title = dict(zip(movies["id"], movies["title"]))
# with open("./features_artifacts/id_to_title.pkl", "wb") as f:
#     pickle.dump(id_to_title, f)

### 4.1 Map IDs (stable indices)

In [65]:
# ratings_df already loaded/cleaned in Step 1
u2i = {u: i for i, u in enumerate(sorted(ratings_df["userId"].unique()))}
# Map MovieLens movieID -> TMDB index from Step 3
links_map= links_cleaned.set_index("movieId")["tmdbId"].to_dict()
tmdb_to_row= {int(t): i for i, t in enumerate(tmdb_ids)} # row index in movie_embeddings

def mlid_to_row(movieId):
    """Map MovieLens movieId to row index in movie_embeddings (or None if not found)."""
    tmdb_id = links_map.get(int(movieId))
    if tmdb_id is None:
        return None
    return tmdb_to_row.get(int(tmdb_id))

ratings_df["user_idx"] = ratings_df["userId"].map(u2i)
ratings_df["movie_idx"] = ratings_df["movieId"].map(mlid_to_row)
ratings_cleaned = ratings_df.dropna(subset=["movie_idx"]).assign(movie_idx=lambda x: x["movie_idx"].astype("int32"))
print("Ratings after cleaning:", ratings_cleaned.shape)
print("Unique users:", ratings_cleaned["userId"].nunique())
print("Unique movies:", ratings_cleaned["movieId"].nunique())

Ratings after cleaning: (99796, 6)
Unique users: 671
Unique movies: 9013


### 4.2 Leave-last-1 per user

In [66]:
if 'timestamp' in ratings_cleaned.columns:
    ratings_cleaned = ratings_cleaned.sort_values(['userId', 'timestamp'])
else:
    ratings_cleaned = ratings_cleaned.sort_values(['userId']) # sort by userId only if timestamp is missing, fallback

test_idx= ratings_cleaned.groupby("userId").tail(1).index
train_df= ratings_cleaned.drop(index=test_idx)
test_df= ratings_cleaned.loc[test_idx]
print("Train ratings:", train_df.shape)
print("Test ratings:", test_df.shape)

Train ratings: (99125, 6)
Test ratings: (671, 6)


### 4.3 Targets

In [67]:
train_df['y'] = (train_df['rating'] >= 4.0).astype('float32')  # threshold at 4.0
test_df['y'] = (test_df['rating'] >= 4.0).astype('float32')  # threshold at 4.0
print("Train positive ratio:", train_df['y'].mean())
print("Test positive ratio:", test_df['y'].mean())

Train positive ratio: 0.51513743
Test positive ratio: 0.55588675


### 5.0 Imports & data shapes


In [68]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

device= "cuda" if torch.cuda.is_available() else "cpu"

num_users= len(u2i)
num_movies= len(tmdb_ids)
print("Number of users:", num_users)
print("Number of movies:", num_movies) 
num_items= movie_embeddings.shape[0] #rows correspond 1:1 to tmdb_ids
emb_dim= movie_embeddings.shape[1] # 128 from content tower
print("Number of items:", num_items)
print("Embedding dimension:", emb_dim)

Number of users: 671
Number of movies: 44383
Number of items: 44383
Embedding dimension: 128


#### 5.1 Dataset & DataLoader

In [69]:
class RatingsDataset(Dataset):
    def __init__(self, df):
        self.user_idx= df["user_idx"].values.astype("int32")
        self.movie_idx= df["movie_idx"].values.astype("int32")
        self.y= df["y"].values.astype("float32")
        # self.N= len(df)

    def __len__(self):
        # return self.N
        return len(self.user_idx)

    def __getitem__(self, i):
        return self.user_idx[i], self.movie_idx[i], self.y[i]
    

train_ds= RatingsDataset(train_df)
test_ds= RatingsDataset(test_df)

trainloader= DataLoader(train_ds, batch_size=1024, shuffle=True, num_workers=0)

#### 5.2 NCF Model (movie emedding initialized from Step 3)

In [70]:
from models.ncf_model import NCF
    
# Stage-1: freeze item embeddings
model= NCF(num_users, num_items, emb_dim, movie_embeddings, freeze_items=True, user_dim=emb_dim).to(device)

#### 5.3 Train (Stage-1 , BCEWithLogitsLoss)

In [71]:
def train_one_epoch(model, loader, opt, loss_fn):
    model.train()
    total, step_loss= 0.0, 0
    for user_idx, movie_idx, y in loader:
        user_idx= user_idx.long().to(device)
        movie_idx= movie_idx.long().to(device)
        y= y.to(device)

        logits= model(user_idx, movie_idx)
        loss= loss_fn(logits, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total += loss.item()
        step_loss += 1
    return total / max(1, step_loss)

loss_fn= nn.BCEWithLogitsLoss()
opt= torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)


In [72]:
# sample = trainloader.dataset[0]
# print(type(sample), sample)


In [73]:
# lower the LR — from 1e-3 → 5e-4 or 1e-4 if there is instability
for epoch in range(3):
    avg_loss= train_one_epoch(model, trainloader, opt, loss_fn)
    print(f"[Stage-1] Epoch {epoch+1} Train Loss BCE: {avg_loss:.4f}")

[Stage-1] Epoch 1 Train Loss BCE: 0.6215
[Stage-1] Epoch 2 Train Loss BCE: 0.6006
[Stage-1] Epoch 3 Train Loss BCE: 0.5957


#### 5.4 Evaluation: Hit@K and NDCG@K (leave-last-1)

In [74]:
train_df.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
user_idx       int64
movie_idx      int32
y            float32
dtype: object

In [75]:
# Build "seen items" per user from training set (to filter out during ranking)
user_seen_items= defaultdict(set)
for u, m in zip(train_df["user_idx"].values, train_df["movie_idx"].values):
    user_seen_items[int(u)].add(int(m))

# Map test users -> (single) test item (leave-last-1)
test_user_to_item= dict(zip(test_df["user_idx"].values, test_df["movie_idx"].values))

print("Test users:", len(test_user_to_item))
print("Example test user->item:", list(test_user_to_item.items())[:5])

Test users: 671
Example test user->item: [(0, 1116), (1, 398), (2, 711), (3, 2320), (4, 3877)]


In [76]:
def rank_for_user(model, user_idx, true_m_idx, K=10, batch_items=4096):
    """Return rank (1-based) of true item among all unseen items. If filtered, returns None."""
    model.eval()
    u= torch.tensor([user_idx], dtype=torch.long, device=device)
    seen_items= user_seen_items.get(user_idx, set())

    # cabdidates = all items excluding seen items
    candidates= np.array([i for i in range(num_items) if i not in seen_items], dtype=np.int32)

    if len(candidates) == 0 or true_m_idx not in candidates:
        return None  # no candidates to rank or true item already seen (edge case)

    scores= []
    with torch.no_grad():
        for i in range(0, len(candidates), batch_items):
            batch = candidates[i:i + batch_items]
            batch_u = u.repeat(len(batch))  # [B]
            batch_m = torch.tensor(batch, dtype=torch.long, device=device)
            logits = model(batch_u, batch_m)
            scores.append(logits.cpu().numpy())
    scores= np.concatenate(scores, axis=0)  # [num_candidates]

    # Rank candidates by score (higher is better)
    ranked_indices= np.argsort(-scores)     # indices of candidates sorted by score desc
    ranked_items= candidates[ranked_indices]  # Numpy array of ranked item indices

    # 1-based rank of true item
    rank= np.where(ranked_items == true_m_idx)[0]
    return int(rank[0]) + 1 if len(rank) > 0 else None

In [77]:
def evaluate_hit_ndcg(model, K=10):
    hits, ndcgs, total= 0, 0.0, 0
    for u, m in test_user_to_item.items():
        rank= rank_for_user(model, u, m, K=K)
        if rank is None:
            continue
        total += 1
        if rank <= K:
            hits += 1
            ndcgs += 1.0 / np.log2(rank + 1)
    hitk= hits / max(1, total)
    ndcgk= ndcgs / max(1, total)
    return hitk, ndcgk, total

hit10, ndcg10, n_eval= evaluate_hit_ndcg(model, K=10)
print(f"[Stage1] Users evaluated: {n_eval}, Hit@10: {hit10:.4f}, NDCG@10: {ndcg10:.4f}")

[Stage1] Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000


#### 5.5 Stage-2: unfreeze items & fine-tune

In [78]:
# Unfreeze item embeddings for fine-tuning
model.item_emb.weight.requires_grad = True
opt= torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)  # lower LR for fine-tuning

for epoch in range(3):
    tr_loss= train_one_epoch(model, trainloader, opt, loss_fn)
    hit10, ndcg10, n_eval= evaluate_hit_ndcg(model, K=10)
    print(f"[Stage-2] Epoch {epoch+1} Train Loss BCE: {tr_loss:.4f} Users evaluated: {n_eval}, Hit@10: {hit10:.4f}, NDCG@10: {ndcg10:.4f}")

[Stage-2] Epoch 1 Train Loss BCE: 0.5671 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
[Stage-2] Epoch 2 Train Loss BCE: 0.5162 Users evaluated: 671, Hit@10: 0.0015, NDCG@10: 0.0005
[Stage-2] Epoch 3 Train Loss BCE: 0.4756 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000


#### 5.6 Save NCF weights

In [79]:
import os
os.makedirs("./checkpoints", exist_ok=True)
torch.save(model.state_dict(), "./checkpoints/ncf_finetuned.pt")
print("Model saved to ./checkpoints/ncf_finetuned.pt")

Model saved to ./checkpoints/ncf_finetuned.pt


### 6. Train & Evaluate the Hybrid

In [80]:
# reuse user_seen_items, test_user_to_item from above
# if missing, rebuild from train_df, test_df
# seen by user= defaultdict(set)
# for u, m in zip(train_df["user_idx"].values, train_df["movie_idx"].values): seen_by_user[int(u)].add(int(m))
# test_user_to_item= dict(zip(test_df["user_idx"].astype(int).values, test_df["movie_idx"].astype(int).values))

#### 6.1 Utilities/ Metrics

In [81]:
def hit_ndcg_at_k(model_fn, test_user_to_item, K=10, batch_items=4096):
    """ model_fn(user_idx, item_idx) -> scores(np.array) or torch.tensor"""
    hits, ndcgs, total= 0, 0.0, 0
    for user_idx, true_m_idx in test_user_to_item.items():

        seen_items= user_seen_items.get(user_idx, set())

        # candidates = all items excluding seen items
        candidates= np.array([i for i in range(num_items) if i not in seen_items], dtype=np.int32)

        if len(candidates) == 0 or true_m_idx not in candidates:
            continue  # no candidates to rank or true item already seen (edge case)

        scores= []

        for i in range(0, len(candidates), batch_items):
            batch = candidates[i:i + batch_items]
            s= model_fn(user_idx, batch)  # np.array or torch.tensor
            if isinstance(s, torch.Tensor):
                s= s.detach().cpu().numpy()
            scores.append(s)
        scores= np.concatenate(scores, axis=0)  # [num_candidates]

        # Rank candidates by score (higher is better)
        ranked_indices= np.argsort(-scores)     # indices of candidates sorted by score desc
        ranked_items= candidates[ranked_indices]  # Numpy array of ranked item indices

        # 1-based rank of true item
        pos= np.where(ranked_items == true_m_idx)[0]
        if pos.size == 0:
            continue
        rank= int(pos[0]) + 1
        total += 1
        if rank <= K:
            hits += 1
            ndcgs += 1.0 / np.log2(rank + 1)

    hitk= hits / max(1, total)
    ndcgk= ndcgs / max(1, total)
    return hitk, ndcgk, total

In [82]:
print(test_df.dtypes)
test_df.head()

userId         int64
movieId        int64
rating       float64
timestamp      int64
user_idx       int64
movie_idx      int32
y            float32
dtype: object


,userId,movieId,rating,timestamp,user_idx,movie_idx,y
4,1,1172,4.0,1260759205,0,1116,1.0
66,2,405,2.0,835356246,1,398,0.0
110,3,736,3.5,1298932787,2,711,0.0
304,4,2454,5.0,949982274,3,2320,1.0
414,5,4025,4.5,1163375145,4,3877,1.0


In [83]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def rmse_mae_on_test(model_pred_fn, test_df):
    """ model_pred_fn(user_idx, movie_idx) -> predicted scores(np.array) or torch.tensor"""
    user= test_df["user_idx"].values.astype("int32")
    movie= test_df["movie_idx"].values.astype("int32")
    y_true= test_df["y"].values.astype("float32") # y =rating
    preds= []
    B=4096
    for i in range(0, len(user), B):
        batch_user= user[i:i+B]
        batch_movie= movie[i:i+B]
        p= model_pred_fn(batch_user, batch_movie)  # np.array or torch.tensor
        if isinstance(p, torch.Tensor):
            p= p.detach().cpu().numpy()
        preds.append(p)
    y_pred= np.concatenate(preds)
    return np.sqrt(mean_squared_error(y_true, y_pred)), mean_absolute_error(y_true, y_pred)

#### 6.2 Content-only scorer (fast dot product)

In [84]:
# content score between user and candidate items
# approach 1 (user profile= mean of seen item embeddings of user)
user_profile_cache= {}
def build_user_profile_mean(user_idx):
    seen = list(user_seen_items.get(user_idx, []))
    if not seen:
        return None
    vec = movie_embeddings[seen].mean(axis=0)
    vec = vec/np.linalg.norm(vec) if np.linalg.norm(vec)>0 else vec # L2-normalize
    return vec

In [85]:
def content_score_user(user_idx, item_idx):
    """ Return array of cos-sim scores for item_indxs"""
    profile= user_profile_cache.get(user_idx)
    if profile is None:
        profile= build_user_profile_mean(user_idx)
        user_profile_cache[user_idx]= profile
    if profile is None:
        # cold users: return popularity or zeros
        return np.zeros(len(item_idx), dtype="float32")  # no profile, return zeros
    return movie_embeddings[item_idx] @ profile  # dot products (cos sim since L2-normalized)

#### 6.3 Collaborative scorer (NCF probabilities)

In [86]:
model.eval()
def ncf_score_user(user_idx, item_idx):
    # batches
    out=[]
    user_tensor= torch.tensor([user_idx], dtype=torch.long, device=device)
    for i in range(0, len(item_idx), 4096):
        batch= item_idx[i:i+4096]
        batch_user= user_tensor.repeat(len(batch))
        batch_item= torch.tensor(batch, dtype=torch.long, device=device)
        with torch.no_grad():
            logits= model(batch_user, batch_item)
        out.append(logits.detach().cpu().numpy())
    return np.concatenate(out).astype("float32")

#### 6.4 Hybrid scorer & alpha tuning

In [87]:
def hybrid_score_user(user_idx, item_idx, alpha):
    """ alpha=0.0 means pure content, alpha=1.0 means pure NCF"""
    c= content_score_user(user_idx, item_idx).astype(np.float32)
    n= ncf_score_user(user_idx, item_idx).astype(np.float32)

    # normalize each vector
    def znorm(x):
        if x.std()==0:
            return x - x.mean()
        return (x - x.mean())/(x.std()+1e-9)
    
    c2= znorm(c)
    n2= znorm(n)

    return alpha * c2 + (1.0-alpha) * n2

In [88]:
# wrapper for evaluator that uses hybrid_score_user
def make_hybrid_fn(alpha):
    def fn(user_idx, item_idx):
        return hybrid_score_user(user_idx, item_idx, alpha)
    return fn

# Grid search over alpha
alphas= np.linspace(0,1,11) # 0.0 .... 1.0
results= []
for alpha in alphas:
    model_fn= make_hybrid_fn(alpha)
    hit10, ndcg10, n_eval= hit_ndcg_at_k(model_fn, test_user_to_item, K=10)
    results.append( (alpha, hit10, ndcg10, n_eval) )
    print(f"Alpha {alpha:.2f} Users evaluated: {n_eval}, Hit@10: {hit10:.4f}, NDCG@10: {ndcg10:.4f}")

# Pick best by Hit or NDCG
best_by_hit= max(results, key=lambda x: x[1])
best_by_ndcg= max(results, key=lambda x: x[2])
print("Best alpha by Hit@10:", best_by_hit)  
print("Best alpha by NDCG@10:", best_by_ndcg)

Alpha 0.00 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
Alpha 0.10 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
Alpha 0.20 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
Alpha 0.30 Users evaluated: 671, Hit@10: 0.0015, NDCG@10: 0.0004
Alpha 0.40 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
Alpha 0.50 Users evaluated: 671, Hit@10: 0.0045, NDCG@10: 0.0014
Alpha 0.60 Users evaluated: 671, Hit@10: 0.0060, NDCG@10: 0.0022
Alpha 0.70 Users evaluated: 671, Hit@10: 0.0060, NDCG@10: 0.0024
Alpha 0.80 Users evaluated: 671, Hit@10: 0.0075, NDCG@10: 0.0028
Alpha 0.90 Users evaluated: 671, Hit@10: 0.0045, NDCG@10: 0.0016
Alpha 1.00 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
Best alpha by Hit@10: (0.8, 0.007451564828614009, 0.0028080878836385615, 671)
Best alpha by NDCG@10: (0.8, 0.007451564828614009, 0.0028080878836385615, 671)


#### 6.5 Offline ranking metrics (Hit@K / NDCG@K) & rating metrics

In [89]:
import scipy.special

# For BCE trained modelm get probability via sigmoid
def ncf_prob_batch(user_idx, movie_idx):
    user_tensor= torch.tensor(user_idx, dtype=torch.long, device=device)
    movie_tensor= torch.tensor(movie_idx, dtype=torch.long, device=device)
    with torch.no_grad():
        logits= model(user_tensor, movie_tensor).detach().cpu().numpy()
    probs= scipy.special.expit(logits)  # sigmoid
    # map prob-> rating e.g. rating= 1 + 4*prob
    return 1.0 + 4.0 * probs

# Compute RMSE/MAE of NCF on test set
rmse, mae= rmse_mae_on_test(ncf_prob_batch, test_df)
print(f"NCF RMSE: {rmse:.4f}, MAE: {mae:.4f}")

NCF RMSE: 2.7981, MAE: 2.5926


#### 6.6 Stage-2 fine-tune

In [90]:
model.item_emb.weight.requires_grad = True
opt= torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-5)  # lower LR for fine-tuning
for epoch in range(3):
    train_loss= train_one_epoch(model, trainloader, opt, loss_fn)
    hit10, ndcg10, n_eval= evaluate_hit_ndcg(model, K=10)
    print(f"[Stage-2b] Epoch {epoch+1} Train Loss BCE: {train_loss:.4f} Users evaluated: {n_eval}, Hit@10: {hit10:.4f}, NDCG@10: {ndcg10:.4f}")

np.savez_compressed("./features_artifacts/movie_embeddings_finetuned.npz", tmdb_id=tmdb_ids, emb=movie_embeddings.astype("float32"))
torch.save(model.state_dict(), "./checkpoints/ncf_finetuned_final.pt")

[Stage-2b] Epoch 1 Train Loss BCE: 0.4043 Users evaluated: 671, Hit@10: 0.0015, NDCG@10: 0.0004
[Stage-2b] Epoch 2 Train Loss BCE: 0.3506 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000
[Stage-2b] Epoch 3 Train Loss BCE: 0.2948 Users evaluated: 671, Hit@10: 0.0000, NDCG@10: 0.0000


#### 6.7 Export final artifacts for serving

In [91]:
# pip install faiss-cpu (or faiss-gpu)

In [92]:
print(movie_embeddings.shape)
print(f"{movie_embeddings.nbytes/1e9:.2f} GB")


(44383, 128)
0.02 GB


In [93]:
import numpy
print(numpy.__version__)
import faiss
print(faiss.__version__)


1.26.4
1.8.0


In [94]:
import faiss
print("FAISS works!")


FAISS works!


In [95]:
import numpy as np
import faiss

x = np.random.rand(1000, 128).astype('float32')
faiss.normalize_L2(x)
index = faiss.IndexFlatIP(128)
index.add(x)
print("Total vectors:", index.ntotal)


Total vectors: 1000


In [96]:
import numpy as np, faiss

# 1) Start from your original matrix (whatever it is named)
X = movie_embeddings  # shape (44383, 128)

# 2) Ensure float32, materialize a brand-new C-contiguous, writeable buffer
X = np.asarray(X, dtype=np.float32, order='C').copy()

# 3) Clean bad rows
#    - Replace NaN/Inf with 0
#    - Remove/repair zero-norm rows to avoid division by zero
bad = ~np.isfinite(X)
if bad.any():
    X[bad] = 0.0

# 4) Safe L2 normalization in NumPy (avoid faiss.normalize_L2 on macOS ARM)
norms = np.linalg.norm(X, axis=1, keepdims=True)
zero_mask = norms.squeeze(-1) == 0.0
# set true zero rows to a tiny vector so norm isn't zero
X[zero_mask] = 0.0
norms = np.linalg.norm(X, axis=1, keepdims=True)
norms[norms == 0.0] = 1.0
X = X / norms

# 5) Build a simple flat IP index and add vectors
d = X.shape[1]
index = faiss.IndexFlatIP(d)
index.add(X)               # should be instant for 44k x 128
print("Total vectors:", index.ntotal)  # expect 44383

# 6) Save with a neutral extension (NOT .ivf for a flat index)
faiss.write_index(index, "./features_artifacts/movie_faiss_index_flat.index")
print("Saved FAISS flat index ✅")


Total vectors: 44383
Saved FAISS flat index ✅


In [97]:
# Kernel crash:
# import numpy as np, faiss

# # Suppose your matrix is named `movie_embeddings`
# arr = np.require(movie_embeddings, dtype=np.float32, requirements=['C','W','A'])
# print(arr.shape, arr.dtype, arr.flags['C_CONTIGUOUS'], arr.flags['WRITEABLE'])

# faiss.normalize_L2(arr)  # in-place normalize; needs C-contiguous & writeable
# idx = faiss.IndexFlatIP(arr.shape[1])
# idx.add(arr)
# print("OK real:", idx.ntotal)  # expect 44383


In [98]:
# Kernel crash:
# import faiss
# d = movie_embeddings.shape[1]
# faiss.normalize_L2(movie_embeddings)
# index = faiss.IndexFlatIP(d)
# index.add(movie_embeddings.astype("float32"))
# faiss.write_index(index, "./features_artifacts/movie_faiss_index_flat.index")
# print("FAISS flat index saved ✅")

### 7. Cold start handling


#### 7.1 Cold movies(no ratings)

In [99]:
print("Seed movie TMDB ID:", tmdb_ids[seed_idx])
# https://www.themoviedb.org/movie/862-toy-story

Seed movie TMDB ID: 862


In [100]:
# Ensure movie_embeddings rows are L2-normalized
def topk_by_content_for_movie(movie_row_idx, k=20):
    query_vec= movie_embeddings[movie_row_idx] # normalized
    sims= movie_embeddings @ query_vec  # cosine similarity (dot product)
    sims[movie_row_idx] = -1.0          # exclude self
    nbrs= np.argpartition(-sims, k)[:k]
    nbrs= nbrs[np.argsort(-sims[nbrs])]
    return nbrs, sims[nbrs]

# Example if movie 0 is “Rush Hour” (Action, Comedy), positives will be any other Action or Comedy movies.
idx, scores= topk_by_content_for_movie(seed_idx, k=10)
print("Neighbors of TMDB ID 862 (Toy Story) by content only:", tmdb_ids[idx], "Scores:", scores)

Neighbors of TMDB ID 862 (Toy Story) by content only: [   863    771 196633   9969  37556  44157  31309  16508  36251  73541] Scores: [0.9179871  0.8985822  0.89654386 0.8953173  0.89306736 0.88762194
 0.8855413  0.88159513 0.8811209  0.8805306 ]


In [101]:
def top_popular_items(k=20):
    stats= ratings_cleaned.groupby("movie_idx")["rating"].agg(["count","mean"])
    # weighted score= count* mean_rating
    stats["weighted"]= stats["count"] * stats["mean"]
    topk= stats.sort_values("weighted", ascending=False).head(k).index.tolist()
    return topk

In [102]:
def build_user_profile_mean(user_idx):
    seen = list(user_seen_items.get(user_idx, []))
    if not seen:
        return None
    vec = movie_embeddings[seen].mean(axis=0)
    norm= np.linalg.norm(vec)
    if norm==0:
        return vec
    return vec/norm # L2-normalize

def recommend_content_for_user(user_idx, k=20):
    profile= build_user_profile_mean(user_idx)
    if profile is None:
        # cold user fallback- popularity or random
        return top_popular_items(k)
    
    sims= movie_embeddings @ profile  # cosine similarity (dot product)
    # filter seen items
    seen_items= user_seen_items.get(user_idx, set())
    sims_list= [(s, i) for i, s in enumerate(sims) if i not in seen_items]
    sims_list.sort(key=lambda x: -x[0])  # sort by score desc
    topk= sims_list[:k]
    return [i for _, i in topk], [s for s, _ in topk]

#### 7.2 Cold movies(few/no ratings)

In [103]:
def alpha_by_num_ratings(n_ratings, C=10.0):
    # when n_ratings=0 -> alpha=1.0 (content-only)
    # as n_ratings increases, alpha -> 0 (collab dominates)
    return float(min(1.0, C / (C + n_ratings)))

def ncf_score_user_array(user_idx, item_idx, batch_size=4096):
    """ Return array of scores for item_indxs"""
    model.eval()
    user_tensor= torch.tensor([user_idx], dtype=torch.long, device=device)
    out=[]
    for i in range(0, len(item_idx), batch_size):
        batch= item_idx[i:i+batch_size]
        batch_user= user_tensor.repeat(len(batch))
        batch_item= torch.tensor(batch, dtype=torch.long, device=device)
        with torch.no_grad():
            logits= model(batch_user, batch_item)
        out.append(logits.detach().cpu().numpy())
    return np.concatenate(out).astype("float32") # shape = (len(item_idxs),)

def zscore(x):
    if x.std()==0:
        return x - x.mean()
    return (x - x.mean())/(x.std()+1e-9)

def hybrid_for_user(user_idx, item_idx, C=10.0):
    # item_idx = list/np.array of candidate item indices to score
    # compute content
    profile= build_user_profile_mean(user_idx)
    if profile is None:
        c_scores= np.zeros(len(item_idx), dtype="float32")  # no profile, return zeros
    else:
        c_scores= movie_embeddings[item_idx] @ profile  # dot products (cos sim since L2-normalized)

    # collaborative
    cf_scores= ncf_score_user_array(user_idx, item_idx)

    # per user normalization
    c2= zscore(c_scores)
    cf2= zscore(cf_scores)

    # compute alpha from user's training count
    n_ratings= len(user_seen_items.get(user_idx, []))
    alpha= alpha_by_num_ratings(n_ratings, C=C) # alpha=1.0 means pure content, alpha=0.0 means pure collab
    scores= alpha * c2 + (1.0 - alpha) * cf2
    return scores, alpha



In [104]:
# pick an external MovieLens userId (e.g., 1)
user_id = 1
u_idx = u2i[user_id]                      # dict lookup → integer index

seen = user_seen_items.get(u_idx, set())     # set of item indices this user has seen

candidates = np.array(
    [i for i in range(movie_embeddings.shape[0]) if i not in seen],
    dtype=np.int32
)

scores, alpha = hybrid_for_user(u_idx, candidates, C=10.0)

order = np.argsort(-scores)[:20]
top_items = candidates[order]

print("Top 20 recommendations for user", user_id, ":", tmdb_ids[top_items], "Alpha:", alpha)


Top 20 recommendations for user 1 : [  9555  10149   5769 183894  12501   1908  25237 133919  17015  14580
  17447 149870  11040  17961  84329  24019  20540  12717  16233 234200] Alpha: 0.3448275862068966


#### 7.3 Cold movies(few/no ratings)

In [105]:
# 7C: Hybrid Fallback Policy (matches your variable names)

# simple alpha schedule for cold users
def alpha_by_num_ratings(n_ratings, C=10.0):
    return float(min(1.0, C / (C + max(0, n_ratings))))

# user profile from content side (mean of seen item embeddings)
def build_user_profile_mean(u_idx):
    seen = list(user_seen_items.get(u_idx, set()))
    if not seen:
        return None
    prof = movie_embeddings[seen].mean(axis=0)
    norm = np.linalg.norm(prof)
    if norm == 0:
        return None
    return prof / norm

# content-only recommenders
def recommend_content_for_user(u_idx, k=20):
    prof = build_user_profile_mean(u_idx)
    if prof is None:
        # true cold user: fallback to popularity
        return np.array(top_popular_items(k), dtype=np.int32), None
    sims = movie_embeddings @ prof
    seen = user_seen_items.get(u_idx, set())
    cand = [(i, sims[i]) for i in range(len(movie_embeddings)) if i not in seen]
    cand.sort(key=lambda x: -x[1])
    top = [i for i,_ in cand[:k]]
    return np.array(top, dtype=np.int32), None

def recommend_neighbors_by_seed_tmdb(seed_tmdb_id, k=20):
    # content neighbors for a given seed movie
    # assumes L2-normalized rows in movie_embeddings
    row = int(np.where(tmdb_ids == seed_tmdb_id)[0][0])
    q = movie_embeddings[row]
    sims = movie_embeddings @ q
    sims[row] = -1.0
    idx = np.argpartition(-sims, k)[:k]
    idx = idx[np.argsort(-sims[idx])]
    return idx, sims[idx]

# main entry: hybrid with fallbacks
def recommend_hybrid(user_id, k=20, C=10.0, seed_tmdb_id=None):
    """
    Policy:
      - If seed_tmdb_id is provided -> content neighbors of that seed (movie→movie).
      - Else if user has no history -> popularity (or content from a cold profile if available).
      - Else -> hybrid blend with alpha based on user's history length.
    Returns: (top_item_indices, scores, alpha_used)
    """
    # seed path (movie-to-movie)
    if seed_tmdb_id is not None:
        idxs, _ = recommend_neighbors_by_seed_tmdb(seed_tmdb_id, k=k)
        return idxs, None, 1.0  # content-only, alpha conceptually 1.0

    # user path
    if user_id not in u2i:
        raise ValueError(f"user_id {user_id} not in mapping")
    u_idx = u2i[user_id]
    seen = user_seen_items.get(u_idx, set())

    # cold user
    if not seen:
        top, _ = recommend_content_for_user(u_idx, k=k)
        # if even profile is None, top is popularity list already
        return top, None, 1.0

    # hybrid user
    candidates = np.array([i for i in range(movie_embeddings.shape[0]) if i not in seen], dtype=np.int32)
    scores, alpha = hybrid_for_user(u_idx, candidates, C=C)  # uses your existing function
    order = np.argsort(-scores)[:k]
    top_items = candidates[order]
    return top_items, scores[order], alpha


#### 7.4 Tune C (alpha schedule) and evaluate on low-history users

In [106]:
import numpy as np

def hit_ndcg_for_low_users(Cs=(1,3,5,10,20), K=10, max_users=None, max_seen=10):
    """
    Evaluate hybrid_for_user on users with few training interactions (<= max_seen),
    sweeping C values. Returns list of tuples: (C, Hit@K, NDCG@K, n_users).
    """
    # select low-history users who also have a test item
    low_users = [u for u in test_user_to_item.keys()
                 if 0 < len(user_seen_items.get(u, set())) <= max_seen]

    if max_users is not None:
        low_users = low_users[:max_users]

    results = []
    for C in Cs:
        hits = 0; ndcg = 0.0; total = 0
        for u_idx in low_users:
            seen = user_seen_items.get(u_idx, set())
            # candidates = all items except seen
            candidates = np.array([i for i in range(movie_embeddings.shape[0]) if i not in seen],
                                  dtype=np.int32)
            if len(candidates) == 0:
                continue

            scores, _alpha = hybrid_for_user(u_idx, candidates, C=C)
            order = np.argsort(-scores)
            ranked = candidates[order]
            true_m = test_user_to_item[u_idx]
            pos = np.where(ranked == true_m)[0]
            if pos.size == 0:
                continue

            r = int(pos[0]) + 1
            total += 1
            if r <= K:
                hits += 1
                ndcg += 1.0 / np.log2(r + 1)

        hitk  = hits / max(1, total)
        ndcgk = ndcg / max(1, total)
        results.append((C, hitk, ndcgk, total))
        print(f"C={C:<4}  Users={total:<4}  Hit@{K}={hitk:.4f}  NDCG@{K}={ndcgk:.4f}")

    best = max(results, key=lambda x: x[2]) if results else None
    if best:
        print("Best (by NDCG):", best)
    else:
        print("No evaluable users found.")
    return results, best

# Try with a slightly looser cutoff first
_7d_results, _7d_best = hit_ndcg_for_low_users(Cs=(1,3,5,10,20), K=10, max_users=None, max_seen=20)


C=1     Users=42    Hit@10=0.0000  NDCG@10=0.0000
C=3     Users=42    Hit@10=0.0000  NDCG@10=0.0000
C=5     Users=42    Hit@10=0.0000  NDCG@10=0.0000
C=10    Users=42    Hit@10=0.0000  NDCG@10=0.0000
C=20    Users=42    Hit@10=0.0000  NDCG@10=0.0000
Best (by NDCG): (1, 0.0, 0.0, 42)


In [107]:
def eval_content_only_low_users(K=10, max_users=None, max_seen=10):
    hits = 0; ndcg = 0.0; total = 0
    low_users = [u for u in test_user_to_item.keys()
                 if 0 < len(user_seen_items.get(u, set())) <= max_seen]
    if max_users: low_users = low_users[:max_users]

    for u_idx in low_users:
        seen = user_seen_items.get(u_idx, set())
        prof = build_user_profile_mean(u_idx)
        if prof is None:
            continue
        sims = movie_embeddings @ prof
        candidates = np.array([i for i in range(movie_embeddings.shape[0]) if i not in seen], dtype=np.int32)
        if len(candidates) == 0:
            continue
        scores = sims[candidates]
        order = np.argsort(-scores)
        ranked = candidates[order]
        true_m = test_user_to_item[u_idx]
        pos = np.where(ranked == true_m)[0]
        if pos.size == 0:
            continue
        r = int(pos[0]) + 1
        total += 1
        if r <= K:
            hits += 1
            ndcg += 1.0 / np.log2(r + 1)
    return (hits / max(1,total), ndcg / max(1,total), total)

content_hit, content_ndcg, content_n = eval_content_only_low_users(K=10, max_seen=20)
print(f"[Low users] Content-only  Users={content_n}  Hit@10={content_hit:.4f}  NDCG@10={content_ndcg:.4f}")


[Low users] Content-only  Users=42  Hit@10=0.0000  NDCG@10=0.0000


##### A) Prep: popularity vector (once)

In [108]:
# # Popularity by item (MovieLens m_idx), min-max normalized to [0,1]
# # Safe if you already computed something similar; redefining is harmless.
# _pop = (
#     ratings_df.groupby("movie_idx")["rating"].count()
#     .reindex(range(movie_embeddings.shape[0]), fill_value=0)
#     .astype("float32")
# ).values

# def _minmax_vec(x):
#     lo, hi = x.min(), x.max()
#     return (x - lo) / (hi - lo + 1e-9) if hi > lo else np.zeros_like(x, dtype=np.float32)

# popularity_norm = _minmax_vec(_pop)  # shape [N_items]


##### B) A min-max variant of your hybrid (optional tiny popularity prior)

In [109]:
# # If you already have these helpers, this will just overwrite with identical behavior.
# def alpha_by_num_ratings(n_ratings, C=10.0):
#     return float(min(1.0, C / (C + max(0, n_ratings))))

# def build_user_profile_mean(u_idx):
#     seen = list(user_seen_items.get(u_idx, set()))
#     if not seen:
#         return None
#     prof = movie_embeddings[seen].mean(axis=0)
#     norm = np.linalg.norm(prof)
#     if norm == 0:
#         return None
#     return prof / norm

# # If you don't already have this defined, this one will suffice.
# def ncf_score_user_array(u_idx, item_idxs, batch_size=4096):
#     model.eval()
#     out = []
#     u_t = torch.tensor([u_idx], dtype=torch.long, device=device)
#     for i in range(0, len(item_idxs), batch_size):
#         batch = item_idxs[i:i+batch_size]
#         uu = u_t.repeat(len(batch))
#         mm = torch.tensor(batch, dtype=torch.long, device=device)
#         with torch.no_grad():
#             logits = model(uu, mm).detach().cpu().numpy()   # higher = more likely
#         out.append(logits.astype(np.float32))
#     return np.concatenate(out, axis=0)

# def hybrid_for_user_minmax(u_idx, item_idxs, C=3.0, pop_lambda=0.05):
#     """
#     Same idea as your hybrid_for_user, but:
#       - per-user MIN-MAX normalize content & collaborative scores
#       - optional small popularity prior (already min-max global)
#     Returns scores, alpha
#     """
#     # content part
#     prof = build_user_profile_mean(u_idx)
#     if prof is None:
#         c_scores = np.zeros(len(item_idxs), dtype=np.float32)
#     else:
#         c_scores = (movie_embeddings[item_idxs] @ prof).astype(np.float32)

#     # collaborative part
#     cf_scores = ncf_score_user_array(u_idx, item_idxs)

#     # per-user normalization (min-max)
#     def mm(x):
#         lo, hi = x.min(), x.max()
#         return (x - lo) / (hi - lo + 1e-9) if hi > lo else np.zeros_like(x, dtype=np.float32)

#     c2  = mm(c_scores)
#     cf2 = mm(cf_scores)

#     # popularity prior on the same candidate set
#     pop2 = popularity_norm[item_idxs]  # already [0,1]

#     # alpha from user’s history size
#     n_ratings = len(user_seen_items.get(u_idx, set()))
#     alpha = alpha_by_num_ratings(n_ratings, C=C)

#     # blend
#     scores = alpha * c2 + (1.0 - alpha) * cf2 + (pop_lambda * pop2)
#     return scores.astype(np.float32), alpha


##### C) Quick compare on low-history users (K=10 and K=20)

In [110]:
# def eval_hybrid_variant_low_users(variant="zscore", K=10, max_seen=10, C=3.0, pop_lambda=0.05):
#     """
#     variant: "zscore" -> uses your existing hybrid_for_user(u, items, C)
#              "minmax" -> uses hybrid_for_user_minmax(u, items, C, pop_lambda)
#     """
#     users = [u for u in test_user_to_item.keys() if 0 < len(user_seen_items.get(u, set())) <= max_seen]
#     hits = 0; ndcg = 0.0; total = 0
#     for u_idx in users:
#         seen = user_seen_items.get(u_idx, set())
#         candidates = np.array([i for i in range(movie_embeddings.shape[0]) if i not in seen], dtype=np.int32)
#         if len(candidates) == 0:
#             continue
#         if variant == "minmax":
#             scores, _ = hybrid_for_user_minmax(u_idx, candidates, C=C, pop_lambda=pop_lambda)
#         else:
#             scores, _ = hybrid_for_user(u_idx, candidates, C=C)  # your existing implementation
#         order = np.argsort(-scores)
#         ranked = candidates[order]
#         true_m = test_user_to_item[u_idx]
#         pos = np.where(ranked == true_m)[0]
#         if pos.size == 0:
#             continue
#         r = int(pos[0]) + 1
#         total += 1
#         if r <= K:
#             hits += 1
#             ndcg += 1.0 / np.log2(r + 1)
#     return (hits / max(1,total), ndcg / max(1,total), total)

# # Run comparisons
# for K in (10, 20):
#     z_hit, z_ndcg, z_n = eval_hybrid_variant_low_users("zscore", K=K, max_seen=10, C=3.0)
#     mm_hit, mm_ndcg, mm_n = eval_hybrid_variant_low_users("minmax", K=K, max_seen=10, C=3.0, pop_lambda=0.05)
#     print(f"[Low users] K={K}  z-score: Users={z_n} Hit@{K}={z_hit:.4f} NDCG@{K}={z_ndcg:.4f}  "
#           f"min-max+pop: Users={mm_n} Hit@{K}={mm_hit:.4f} NDCG@{K}={mm_ndcg:.4f}")


In [111]:
# import numpy as np

# # 1) Rebuild mappings (train → seen, test → held-out), minimal & consistent
# user_seen_items = {}
# for u, m in zip(train_df["user_idx"].astype(int).values, train_df["movie_idx"].astype(int).values):
#     user_seen_items.setdefault(u, set()).add(m)

# test_user_to_item = dict(
#     zip(test_df["user_idx"].astype(int).values, test_df["movie_idx"].astype(int).values)
# )

# # 2) Quick diagnostics
# users_with_test = list(test_user_to_item.keys())
# low_users_10 = [u for u in users_with_test if 0 < len(user_seen_items.get(u, set())) <= 10]
# low_users_20 = [u for u in users_with_test if 0 < len(user_seen_items.get(u, set())) <= 20]

# print(f"Users with test: {len(users_with_test)} | low<=10: {len(low_users_10)} | low<=20: {len(low_users_20)}")

# # Count problematic users where the true test item is somehow already in 'seen' (should be 0 for leave-last-1)
# problem = sum(1 for u in users_with_test if test_user_to_item[u] in user_seen_items.get(u, set()))
# print("Users with true test item mistakenly in 'seen':", problem)

# # 3) Minimal evaluator that uses YOUR hybrid_for_user and fixes only when necessary
# def eval_hybrid_existing(K=10, max_seen=10, C=3.0):
#     users = [u for u in users_with_test if 0 < len(user_seen_items.get(u, set())) <= max_seen]
#     hits = 0; ndcg = 0.0; total = 0
#     for u_idx in users:
#         seen = user_seen_items.get(u_idx, set())
#         true_m = test_user_to_item[u_idx]

#         # candidates = all items except seen; ensure the true item is present
#         candidates = np.array([i for i in range(movie_embeddings.shape[0]) if i not in seen], dtype=np.int32)
#         if candidates.size == 0:
#             continue
#         if true_m not in candidates:
#             # If this happens due to a prev split glitch, include it so we can still evaluate.
#             candidates = np.append(candidates, true_m)

#         scores, _alpha = hybrid_for_user(u_idx, candidates, C=C)
#         order = np.argsort(-scores)
#         ranked = candidates[order]
#         pos = np.where(ranked == true_m)[0]
#         if pos.size == 0:
#             continue
#         r = int(pos[0]) + 1
#         total += 1
#         if r <= K:
#             hits += 1
#             ndcg += 1.0 / np.log2(r + 1)

#     hitk = hits / max(1, total)
#     ndcgk = ndcg / max(1, total)
#     return hitk, ndcgk, total

# # 4) Run with your existing hybrid_for_user (no extra functions), K=10 and K=20
# for K in (10, 20):
#     hit, ndcg, n = eval_hybrid_existing(K=K, max_seen=10, C=3.0)
#     print(f"[Low users<=10] K={K}: Users={n}  Hit@{K}={hit:.4f}  NDCG@{K}={ndcg:.4f}")

# # If the <=10 slice is tiny, also show <=20 quickly:
# for K in (10, 20):
#     hit, ndcg, n = eval_hybrid_existing(K=K, max_seen=20, C=3.0)
#     print(f"[Low users<=20] K={K}: Users={n}  Hit@{K}={hit:.4f}  NDCG@{K}={ndcg:.4f}")


#### 7.5: Serving: FAISS shortlist → hybrid re-rank

In [112]:
LOW_HISTORY_MAX_SEEN = 20
C_FOR_LOW_USERS = 3.0
TOPK_TO_RETURN = 20  # or 10 if you prefer

In [113]:
print("shape:", movie_embeddings.shape,
      "dtype:", movie_embeddings.dtype,
      "contiguous:", movie_embeddings.flags['C_CONTIGUOUS'],
      "finite:", np.isfinite(movie_embeddings).all())


shape: (44383, 128) dtype: float32 contiguous: True finite: True


In [114]:
# ---- 7E (robust): FAISS shortlist + hybrid re-rank with safe normalization ----
import numpy as np
import faiss

def _safe_l2_normalize_rows(mat: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    # Ensure float32, C-contiguous, finite, no zero norms
    mat = np.asarray(mat, dtype=np.float32, order="C")
    np.nan_to_num(mat, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    norms = np.linalg.norm(mat, axis=1, keepdims=True)
    norms = np.maximum(norms, eps)
    mat = mat / norms
    return mat

def _safe_l2_normalize_vec(v: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    v = np.asarray(v, dtype=np.float32).reshape(1, -1)
    np.nan_to_num(v, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    n = np.linalg.norm(v, axis=1, keepdims=True)
    n = np.maximum(n, eps)
    return (v / n).astype(np.float32)

# 0) Prepare embeddings safely
movie_embeddings = _safe_l2_normalize_rows(movie_embeddings)
tmdb_ids = np.asarray(tmdb_ids)  # ensure np array

# 1) Build FAISS IP index (cosine if rows are L2-normalized)
d = movie_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(d)
faiss_index.add(movie_embeddings.astype("float32", copy=False))
print("FAISS index ready. Vectors:", faiss_index.ntotal, "Dim:", d)

# 2) Shortlist helpers
def shortlist_by_content_for_user(u_idx, M=200):
    """
    Use the user's content profile to pull a shortlist via FAISS.
    Falls back to simple unseen-scan if the user is totally cold.
    """
    prof = build_user_profile_mean(u_idx)
    seen = user_seen_items.get(u_idx, set())

    if prof is None:
        # fallback: first M unseen (deterministic & safe)
        if len(seen) >= movie_embeddings.shape[0]:
            return np.array([], dtype=np.int32)
        idxs = [i for i in range(movie_embeddings.shape[0]) if i not in seen][:M]
        return np.array(idxs, dtype=np.int32)

    q = _safe_l2_normalize_vec(prof)  # (1, d) float32 normalized
    sims, idxs = faiss_index.search(q, M)  # idxs: (1, M)
    idxs = idxs[0]
    # filter out seen
    if len(seen) > 0:
        idxs = np.array([i for i in idxs if i not in seen], dtype=np.int32)
    else:
        idxs = idxs.astype(np.int32, copy=False)
    return idxs

def shortlist_by_content_seed(seed_tmdb_id, M=200):
    """
    Movie→movie: FAISS neighbors of a seed movie (content-only).
    """
    seed_pos = np.where(tmdb_ids == seed_tmdb_id)[0]
    if seed_pos.size == 0:
        raise ValueError(f"Seed TMDB id {seed_tmdb_id} not found.")
    row = int(seed_pos[0])

    q = _safe_l2_normalize_vec(movie_embeddings[row])
    sims, idxs = faiss_index.search(q, M)
    idxs = idxs[0]
    # remove self if present
    idxs = idxs[idxs != row]
    return idxs.astype(np.int32, copy=False)

# 3) Main entry: shortlist → hybrid re-rank
def recommend_hybrid_with_faiss(user_id, k=TOPK_TO_RETURN, M=200, C=None, seed_tmdb_id=None):
    """
    Two-stage serving:
      1) FAISS shortlist by content (user profile OR seed movie)
      2) Hybrid re-rank the shortlist with your existing hybrid_for_user (z-score)
    Returns: (top_item_indices, scores_or_None, alpha_used)
    """
    if C is None:
        C = C_FOR_LOW_USERS

    # Seed path (movie-to-movie content recommendations)
    if seed_tmdb_id is not None:
        shortlist = shortlist_by_content_seed(seed_tmdb_id, M=M)
        return shortlist[:k], None, 1.0

    # User path
    if user_id not in u2i:
        raise ValueError(f"user_id {user_id} not in mapping")
    u_idx = u2i[user_id]

    shortlist = shortlist_by_content_for_user(u_idx, M=M)
    if shortlist.size == 0:
        return np.array([], dtype=np.int32), None, 1.0

    # hybrid re-rank on shortlist
    scores, alpha = hybrid_for_user(u_idx, shortlist, C=C)
    order = np.argsort(-scores)[:k]
    topk = shortlist[order]
    return topk, scores[order], alpha

# 4) Examples
# A) user recommendations (hybrid)
topk, scores, alpha = recommend_hybrid_with_faiss(user_id=1, k=TOPK_TO_RETURN, M=200, C=C_FOR_LOW_USERS)
print("User 1 → TMDB:", tmdb_ids[topk], "alpha:", alpha)

# B) movie→movie neighbors (content-only)
neighbors, _, _ = recommend_hybrid_with_faiss(user_id=1, seed_tmdb_id=862, k=10, M=200)
print("Neighbors of 862 → TMDB:", tmdb_ids[neighbors])


FAISS index ready. Vectors: 44383 Dim: 128
User 1 → TMDB: [ 27022  31606  39314   9662  27205  41800     98 118490  11712  10856
 266639 339967 128048 135397   1406  27957  31501   2926  57706  21332] alpha: 0.13636363636363635
Neighbors of 862 → TMDB: [   863    771 196633   9969  37556  44157  31309  16508  36251  73541]


In [115]:
# === Export mappings for the API (run after Step 3 & 4) ===
# Requires in memory:
# - train_df (from your Step 4 split, with user/item indices)
# - movie_embeddings, tmdb_ids (from your Step 3/6 content tower)
# - a movies-like DataFrame with columns ['id','title'] (e.g., metadata_cleaned or movies)

import os, json, pickle
import numpy as np
import pandas as pd

os.makedirs("./mappings", exist_ok=True)

# ---- 1) Find the right column names in train_df ----
def _pick(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

if 'train_df' not in globals():
    raise RuntimeError("train_df not found. Run your Step 4 split first.")

user_ext_col = _pick(train_df, ("userId", "user_id", "userid"))
user_idx_col = _pick(train_df, ("u_idx", "user_idx", "user_index"))
movie_idx_col = _pick(train_df, ("m_idx", "movie_idx", "item_idx", "movie_index"))

if not all([user_ext_col, user_idx_col, movie_idx_col]):
    raise RuntimeError(
        f"Could not find needed columns in train_df. "
        f"Found user_ext_col={user_ext_col}, user_idx_col={user_idx_col}, movie_idx_col={movie_idx_col}. "
        f"Open train_df.head() to confirm column names."
    )

# ---- 2) Build u2i: external MovieLens userId -> internal user index (u_idx) ----
tmp = train_df[[user_ext_col, user_idx_col]].drop_duplicates()
u2i = {int(r[user_ext_col]): int(r[user_idx_col]) for _, r in tmp.iterrows()}

# ---- 3) Build user_seen_items from TRAIN: u_idx -> set of m_idx ----
user_seen_items = {}
for u, m in zip(train_df[user_idx_col].astype(int).values,
                train_df[movie_idx_col].astype(int).values):
    user_seen_items.setdefault(int(u), set()).add(int(m))

# ---- 4) Build id_to_title (TMDB id -> title) filtered to IDs present in embeddings ----
# Try to auto-pick a movies dataframe with ['id','title'] columns
movies_df = None
for cand in ("movies", "metadata_cleaned", "metadata", "movies_metadata"):
    if cand in globals():
        df = globals()[cand]
        if isinstance(df, pd.DataFrame) and {"id","title"}.issubset(df.columns):
            movies_df = df[["id","title"]].dropna()
            break

id_to_title = {}
if movies_df is not None:
    tmdb_array = np.asarray(tmdb_ids)  # must exist from Step 3/6
    tmdb_set = set(int(x) for x in tmdb_array.tolist())
    sub = movies_df[movies_df["id"].astype(int).isin(tmdb_set)]
    id_to_title = {int(r.id): str(r.title) for _, r in sub.iterrows()}

# ---- 5) Write files (overwrites old ones) ----
with open("./mappings/u2i.json", "w", encoding="utf-8") as f:
    json.dump({int(k): int(v) for k, v in u2i.items()}, f, ensure_ascii=False)

with open("./mappings/user_seen_items.pkl", "wb") as f:
    pickle.dump({int(u): set(int(m) for m in s) for u, s in user_seen_items.items()}, f)

if id_to_title:
    with open("./mappings/id_to_title.json", "w", encoding="utf-8") as f:
        json.dump(id_to_title, f, ensure_ascii=False)

print(f"Wrote u2i.json ({len(u2i)} users), user_seen_items.pkl ({len(user_seen_items)} users), "
      f"id_to_title.json ({len(id_to_title)} titles).")
print("Sample user IDs:", list(sorted(u2i.keys()))[:8])


Wrote u2i.json (671 users), user_seen_items.pkl (671 users), id_to_title.json (44383 titles).
Sample user IDs: [1, 2, 3, 4, 5, 6, 7, 8]
